## Dataset generation from deformable mirror and GRIN fiber

In [5]:
from lib.grid import Grid
from lib.fiber import GrinFiber
from lib.datasets import SimulatedGrinSpeckleOutputDataset

In [6]:
grid = Grid(pixel_size=0.5e-6)
fiber = GrinFiber(radius=26e-6, wavelength=1064e-9, n1=1.465, n2=1.45)

dset = SimulatedGrinSpeckleOutputDataset(fiber, grid, length=100, N_modes=55)
dset.compute(phases_dim=(6,6))
dset.export()

NameError: name 'fiber' is not defined